In [ ]:
# https://joint-research-centre.ec.europa.eu/pvgis-online-tool/getting-started-pvgis/api-non-interactive-service_en

import os, csv, json, requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from zipfile import ZipFile
from IPython.display import clear_output

from pathlib import Path
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
__author__  = "Fernando Fahl <fernando.fahl@gmail.com>"
__version__ = "1.1"
__date__    = "March 2023"

# 1. Parameters

## 1.1 Input

In [ ]:
# ____________ API parameters
startyear     = 2020
endyear       = 2020
peakpower     = 1
loss          = 14
pvcalculation = 1 # "0" outputs only solar radiation calculations, "1" outputs the estimation of hourly PV production as well
optimalangles = 1 #  Value of 1 for "yes". All other values (or no value) mean "no". 

# ____________ location points (from excel file): required fields
excel_filename = 'EU_random_points.xlsx'
column_id      = 'id'
column_lat     = 'latitude'
column_lon     = 'longitude'
column_timezone = 'time_zone' # time zone must be one of the columns. In case of no time zone transformation, use 0

# ____________ compression output file (True=yes, False=No)
compress_output = False

# 2. Data load

In [ ]:
today       = date.today().isoformat()
home        = Path(os.getcwd())
xls_file    = home.joinpath(excel_filename)

csv_outfile = home.joinpath(f'{xls_file.stem}_year{startyear}to{endyear}_{today}.csv')
csv_errors  = home.joinpath(f'{xls_file.stem}_ERRORs_{today}.csv')

print (csv_outfile)

In [ ]:
df_xls = pd.read_excel(xls_file)
columns = [column_id,column_lat,column_lon,column_timezone]
df = df_xls[columns].copy()
df.set_index(column_id, inplace=True)
print(df.tail())

# 3. PVGIS

## 3.1 Create URL

In [ ]:
def get_url(lat, lon):    
    
    url_base = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?"
    
    pvgis_params = dict(
        peakpower=peakpower,
        loss=loss,
        startyear=startyear,
        endyear=endyear,
        pvcalculation=pvcalculation, # "0" outputs only solar radiation calculations, "1" outputs the estimation of hourly PV production as well
        optimalangles=optimalangles,
        lat=lat,
        lon=lon,
        outputformat = 'json',
    )   

    return url_base + "&".join([f'{key}={value}' for key, value in pvgis_params.items()])


## 3.2 Parse PVGIS

In [ ]:
def parse_json(id, data, time_zone, startyear, endyear):    
    
    # ____________ parse data to df
    df_input        = pd.json_normalize(data.get('inputs'))
    df_output       = pd.json_normalize(data.get('outputs').get('hourly'))        
    latitude        = df_input['location.latitude'].values
    
    # ____________ get optimal angles
    slope_optimized   = df_input['mounting_system.fixed.slope.value'].values
    azimuth_optimized = df_input['mounting_system.fixed.azimuth.value'].values    
    azimuth_corrected = 0 if float(latitude) < 0 else 180

    # ____________ convert the 'Date' column to datetime format: round minutes from timestamp (do not trunc the timestamp - it would cause a shift in some readings). problems to round hours when it is .30 (generates duplicates) - solution below solves the problem
    df_power               = df_output[['time', 'P']].copy() 
    df_power['time']       = pd.to_datetime(df_power['time'], format='%Y%m%d:%H%M')
    df_power['nearest_hour'] = df_power["time"].dt.ceil("H").where(df_power["time"].dt.minute > 30, df_power["time"].dt.floor("H"))
    df_power['local_time'] = df_power['nearest_hour'] + timedelta(hours = time_zone)
    
    # ____________ reformat timestamp (remove minutes)
    df_power['local_time']  = df_power['local_time'].dt.strftime('%Y-%m-%d:%H')    
    
    # ____________ select data for the data range (time shift generates data for the next year for positive zones and previous year for negative zones)   
    date_start = f'{startyear}-01-01:00'
    date_end   = f'{endyear}-12-31:23'    
    df_mask    = df_power.loc[df_power['local_time'].between(date_start, date_end, inclusive='both')]
    
    # # ____________ fill missing data (it can give error if 
    # df_mask = df_mask.set_index(df_mask['local_time'])
    # new_date_range = pd.date_range(start=datetime.strptime(date_start, '%Y-%m-%d:%H'), end=datetime.strptime(date_end, '%Y-%m-%d:%H'), freq="H").strftime('%Y-%m-%d:%H')
    # df_mask.reindex(new_date_range, fill_value=0)
     
    # ____________ set datetime index
    df_mask = df_mask.set_index(df_mask['local_time'])
    df_mask = df_mask.drop(['time', 'local_time'], axis=1)   
    
    # ____________ transpose df
    df_tranpose = df_mask[['P']].transpose()   
    
    # ____________ add columns
    df_tranpose.insert(0, 'time_zone', time_zone)
    df_tranpose.insert(0, 'azimuth_corrected', azimuth_corrected)
    df_tranpose.insert(0, 'azimuth', azimuth_optimized)
    df_tranpose.insert(0, 'slope', slope_optimized)
    df_tranpose.insert(0, 'id', id)   
    
    return df_tranpose    
    

## 3.3 Query PVGIS

In [ ]:
def save_csv(dfs, dfs_error):
    """ save csv as compressed files"""
    
    if compress_output:
        compression='gzip'
        out_name=f"{csv_outfile}.gz"
        out_name_error=f"{csv_errors}.gz"
    else:
        compression=None
        out_name=csv_outfile
        out_name_error=csv_errors

    df_merged = pd.concat(dfs.values(), axis=0, ignore_index=True)       
    df_merged.to_csv(out_name, index = False, compression='gzip')
    print (f"\n...saving file: {out_name}")
    
    if dfs_error:    
        df_merged_error = pd.concat(dfs_error.values(), axis=0, ignore_index=True)    
        df_merged_error.to_csv(out_name_error, index = False, compression='gzip')  
        print (f"...saving file: {out_name_error}")      

In [ ]:
i = 0
dfs = dict()
dfs_error = dict()

for idx, row in df.iterrows(): 
    
    id   = idx
    lat  = row[column_lat].astype(str)
    lon  = row[column_lon].astype(str)
    time_zone = int(row[column_timezone])
    url  = get_url (lat,lon)

    # _____________ get data from pvgis
    response = requests.get(url)
    row_json = json.loads(response.text)    

    # _____________ parse data from pvgis
    try:            
        dfs[id] = parse_json(id, row_json, time_zone, startyear, endyear)
        print (f'done {i}: id={id} time zone={time_zone} url={url}')
    except:
        message = row_json['message']
        dfs_error[id] = pd.DataFrame(dict(id=id,lat=lat,lon=lon,message=message,url=url), index=[0])
        print (f'ERROR {i} --> id: {id}, message: {message} url: {url}')

    # _____________ save partial results (if it contains more than 500 points)
    if i > 500:  
        save_csv(dfs, dfs_error)        
        i = 0
        clear_output(wait=False)  
    i +=1      
            
# _____________ save final results
save_csv(dfs, dfs_error)  
